# German Basic Laws Chatbot

## Import all the libraries here

In [1]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from pypdf import PdfReader
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
load_dotenv()
import openai

## Read the files

**Document: The Basic Laws of the Federal Republic of Germany**
- This document contains the basic laws of the Federal Republic of Germany.
- It is an open source document.
- Source: [German Bundestag](btg-bestellservice.de>tcpdf)

In [2]:
document_dir = "../data"
file_names = os.listdir(document_dir)

# read the files and put them into strings
text = ""
for file in file_names:
    file_path = os.path.join(document_dir, file)
    print(file)
    reader = PdfReader(file_path)
    for page in reader.pages:
        text += page.extract_text() + "\n"

german_basic_law.pdf
german_fiscal_law.pdf
german_civil_law.pdf
german_commercial_law.pdf
german_criminal_law.pdf
german_administrative_law.pdf
german_employment_law.pdf
german_family_law.pdf
german_alcohol_law.pdf


In [3]:
len(text)

4554620

In [4]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap = 200
)

chunks = text_splitter.split_text(text)

In [5]:
len(chunks)

5743

In [6]:
chunks[:10]

['Basic Law \nfor the Federal Republic \nof Germany \n23 May 1949 \nLast amended on 19 December 2022\nHerausgeber: Deutscher Bundestag,\nReferat Öffentlichkeitsarbeit,\nPlatz der Republik 1, 11011 Berlin\nKoordination: Elmar Ostermann\nPublished by: German Bundestag \nPublic Relations Division \nPlatz der Republik 1, 11011 Berlin \nCoordination: Elmar Ostermann\nTranslated by: Professor Christian Tomuschat, Professor David P. Currie, \nProfessor Donald P. Kommers and Raymond Kerr, in cooperation with the \nLanguage Service of the German Bundestag\nDesign: Regelindis Westphal Grafik-Design,  \nedited by wbv Media, Christiane Zay\nBundestag eagle: Created by Professor Ludwig Gies,  \nrevised in 2008 by büro uebele\n© Deutscher Bundestag, Berlin, June 2024\nAll rights reserved.\nThis publication has been produced as part of the German Bundestag’s \npublic relations activities. It is provided free of charge and is not intended \nfor sale. It may not be used for election campaign purposes o

## Embeddings

In [7]:
# using openai embeddings
api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

/tmp/ipykernel_3568/840955190.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


## Create Chroma DB

In [8]:
db = Chroma.from_texts(
    chunks,
    embeddings,
    persist_directory="./chroma_db"
)
print("ChromaDB created with text embeddings.")

ChromaDB created with text embeddings.


## Retrieving Documents

In [9]:
user_question = "What is fundamental human rights?"
retrieved_docs = db.similarity_search(user_question, k=10)

In [10]:
retrieved_docs

[Document(metadata={}, page_content='it shall be the duty of all state authority.\n (2) The German people therefore acknowledge inviolable and \ninalienable human rights as the basis of every community, \nof peace and of justice in the world.\n (3) The following basic rights shall bind the legislature, the \nexecutive and the judiciary as directly applicable law. \nArticle 2 \n[Personal freedoms]\n (1) Every person shall have the right to free development of his \npersonality insofar as he does not violate the rights of oth-\ners or offend against the constitutional order or the moral \nlaw.\n (2) Every person shall have the right to life and physical \nintegrity. Freedom of the person shall be inviolable. These \nrights may be interfered with only pursuant to a law. \nArticle 3 \n[Equality before the law]\n (1) All persons shall be equal before the law.\n (2) Men and women shall have equal rights. The state shall \npromote the actual implementation of equal rights for wom-\nen and men

In [11]:
# Display top results
for i, doc in enumerate(retrieved_docs): # Display top 3 results
    print(f"Document {i+1}:\n{doc.page_content}") # Display content

Document 1:
it shall be the duty of all state authority.
 (2) The German people therefore acknowledge inviolable and 
inalienable human rights as the basis of every community, 
of peace and of justice in the world.
 (3) The following basic rights shall bind the legislature, the 
executive and the judiciary as directly applicable law. 
Article 2 
[Personal freedoms]
 (1) Every person shall have the right to free development of his 
personality insofar as he does not violate the rights of oth-
ers or offend against the constitutional order or the moral 
law.
 (2) Every person shall have the right to life and physical 
integrity. Freedom of the person shall be inviolable. These 
rights may be interfered with only pursuant to a law. 
Article 3 
[Equality before the law]
 (1) All persons shall be equal before the law.
 (2) Men and women shall have equal rights. The state shall 
promote the actual implementation of equal rights for wom-
en and men and take steps to eliminate disadvantages th

In [12]:
def _get_document_prompt(docs):
    prompt = "\n"
    for doc in docs:
        prompt += "\nContent:\n"
        prompt += doc.page_content + "\n\n"
    return prompt

In [13]:
formatted_context = _get_document_prompt(retrieved_docs)

In [14]:
formatted_context

'\n\nContent:\nit shall be the duty of all state authority.\n (2) The German people therefore acknowledge inviolable and \ninalienable human rights as the basis of every community, \nof peace and of justice in the world.\n (3) The following basic rights shall bind the legislature, the \nexecutive and the judiciary as directly applicable law. \nArticle 2 \n[Personal freedoms]\n (1) Every person shall have the right to free development of his \npersonality insofar as he does not violate the rights of oth-\ners or offend against the constitutional order or the moral \nlaw.\n (2) Every person shall have the right to life and physical \nintegrity. Freedom of the person shall be inviolable. These \nrights may be interfered with only pursuant to a law. \nArticle 3 \n[Equality before the law]\n (1) All persons shall be equal before the law.\n (2) Men and women shall have equal rights. The state shall \npromote the actual implementation of equal rights for wom-\nen and men and take steps to eli

## Chatbot Architechture

In [15]:
prompt = f"""
## SYSTEM ROLE
You are a knowledgeable and factual chatbot designed to assist with technical and theoretical questions about **German Basic Law**. 
Your answers must be based exclusively on provided content from technical books provided.

## USER QUESTION
The user has asked: 
"{user_question}"

## CONTEXT
Here is the relevant content from the article:  
'''
{formatted_context}
'''

## GUIDELINES
1. **Accuracy**:  
   - Only use the content in the `CONTEXT` section to answer.  
   - If the answer cannot be found, explicitly state: "The provided context does not contain this information."

2. **Transparency**:  
   - Reference the article's name and page numbers when providing information.  
   - Do not speculate or provide opinions.  

3. **Clarity**:  
   - Use simple, professional, and concise language.  
   - Format your response in Markdown for readability.  

## TASK
1. Answer the user's question **directly** if possible.  
2. Point the user to relevant parts of the documentation.  
3. Provide the response in the following format:

## RESPONSE FORMAT
'''
# [Brief Title of the Answer]
[Answer in simple, clear text.]

**Source**:  
• [Article Title], Page(s): [...]
'''
"""
print("Prompt constructed.")

Prompt constructed.


In [16]:
# Set up GPT client and parameters
client = openai.OpenAI()
model_params = {
    'model': 'gpt-4o',
    'temperature': 0.7,  # Increase creativity
    'max_tokens': 4000,  # Allow for longer responses
    'top_p': 0.9,        # Use nucleus sampling
    'frequency_penalty': 0.5,  # Reduce repetition
    'presence_penalty': 0.6    # Encourage new topics
}

## Response

In [17]:
messages = [{'role': 'user', 'content': prompt}]
completion = client.chat.completions.create(messages=messages, **model_params, timeout=120)

In [18]:
answer = completion.choices[0].message.content
print(answer)

'''
# Fundamental Human Rights in German Basic Law

Fundamental human rights are recognized as inviolable and inalienable by the German Basic Law. They form the basis of every community, peace, and justice in the world. The Basic Law directly binds the legislature, executive, and judiciary to these basic rights.

**Source**:  
• Article 1 [Human dignity – Human rights – Legally binding force of basic rights], Page(s): 15
'''


'''
# Fundamental Human Rights in the German Basic Law

Fundamental human rights, as acknowledged by the German Basic Law, are described as inviolable and inalienable rights that form the basis of every community, peace, and justice in the world. These rights bind the legislature, executive, and judiciary as directly applicable law.

**Source**:  
• Article 1 [Human dignity – Human rights – Legally binding force of basic rights], Page(s): 15
'''

In [19]:
from sentence_transformers import SentenceTransformer

In [47]:
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[ 6.76569492e-02  6.34959266e-02  4.87131327e-02  7.93049708e-02
   3.74480784e-02  2.65277107e-03  3.93749774e-02 -7.09846988e-03
   5.93614690e-02  3.15370075e-02  6.00980744e-02 -5.29051870e-02
   4.06067856e-02 -2.59308480e-02  2.98427902e-02  1.12688087e-03
   7.35148862e-02 -5.03818430e-02 -1.22386619e-01  2.37028226e-02
   2.97265742e-02  4.24768738e-02  2.56337207e-02  1.99517375e-03
  -5.69190271e-02 -2.71598082e-02 -3.29035670e-02  6.60249069e-02
   1.19007148e-01 -4.58791256e-02 -7.26214945e-02 -3.25840078e-02
   5.23413271e-02  4.50553373e-02  8.25297926e-03  3.67023870e-02
  -1.39415376e-02  6.53918535e-02 -2.64272355e-02  2.06400568e-04
  -1.36643825e-02 -3.62810604e-02 -1.95043404e-02 -2.89738271e-02
   3.94270681e-02 -8.84090438e-02  2.62427493e-03  1.36714075e-02
   4.83062714e-02 -3.11565809e-02 -1.17329195e-01 -5.11690080e-02
  -8.85287821e-02 -2.18962636e-02  1.42986281e-02  4.44167145e-02
  -1.34815276e-02  7.43392706e-02  2.66382638e-02 -1.98762398e-02
   1.79191